In [2]:
from dotenv import find_dotenv,load_dotenv
from pprint import pprint
load_dotenv(find_dotenv(), override=True)
import os

os.environ["LANGCHAIN_PROJECT"]="raglanggraph"

In [3]:
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import StructuredTool

In [41]:
from typing import TypedDict, Annotated, List, Union
from langchain_core.agents import AgentAction, AgentFinish
from langchain_core.messages import BaseMessage
import operator

class AgentState(TypedDict):
    input: str
    chat_history: list[BaseMessage]
    agent_outcome: Union[AgentAction, AgentFinish, None]
    intermediate_steps: Annotated[list[tuple[AgentAction, str]], operator.add]

In [32]:
from langchain.tools import BaseTool, StructuredTool, Tool, tool
import random

@tool("lower_case", return_direct=True)
def to_lower_case(input: str)->str:
    """Returns the input as all lower case"""
    return input.lower()

@tool("random_number", return_direct=True)
def random_number_maker(input:str)->str:
    """Returns a random number between 0-100"""
    return random.randint(1,100)

tools = [to_lower_case, random_number_maker]

In [33]:
from langchain  import hub
from langchain.agents import create_openai_functions_agent
from langchain_openai.chat_models import ChatOpenAI

prompt = hub.pull("hwchase17/openai-functions-agent")

llm = ChatOpenAI(model="gpt-3.5-turbo-1106", streaming=True)

agent_runnable = create_openai_functions_agent(
    llm,
    tools,
    prompt
)

In [34]:
inputs = {
    "input": "Give me a random number and then write in word to make it lower case.",
    "chat_history": [],
    "intermediate_steps": []
}

In [35]:
agent_outcome = agent_runnable.invoke(inputs)

In [37]:
from langgraph.prebuilt.tool_executor import ToolExecutor

tool_executor = ToolExecutor(tools)

In [38]:
def run_agent(data):
    agent_outcome = agent_runnable.invoke(data)
    return {"agent_outcome": agent_outcome}

def execute_tools(data):
    agent_action = data["agent_outcome"]
    output = tool_executor.invoke(agent_action)
    print(f"agent action is {agent_action}")
    print(f"The tool result is: {output}")
    return {"intermediate_steps": [(agent_action, str(output))]}

def should_continue(data):
    
    if(isinstance(data['agent_outcome'], AgentFinish)):
        return "end"
    else:
        return "continue"

In [42]:
from langgraph.graph import END, StateGraph

workflow = StateGraph(AgentState)

workflow.add_node("agent", run_agent)
workflow.add_node("action", execute_tools)

workflow.set_entry_point("agent")

workflow.add_conditional_edges(
    "agent",
    should_continue,
    {
        "continue": "action",
        "end": END
    }
)

workflow.add_edge("action","agent")

app = workflow.compile()


In [43]:
input = {
    "input": "give me a random number and them write in words and make it lower case", "chat_history": []
}

In [44]:
output = app.invoke(input)

agent action is tool='random_number' tool_input={'input': 'random number'} log="\nInvoking: `random_number` with `{'input': 'random number'}`\n\n\n" message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"input":"random number"}', 'name': 'random_number'}}, response_metadata={'finish_reason': 'function_call'}, id='run-e1ece6be-900d-4eef-bff7-7063e0a0e947-0')]
The tool result is: 55
agent action is tool='lower_case' tool_input={'input': 'FIFTY-FIVE'} log="\nInvoking: `lower_case` with `{'input': 'FIFTY-FIVE'}`\n\n\n" message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"input":"FIFTY-FIVE"}', 'name': 'lower_case'}}, response_metadata={'finish_reason': 'function_call'}, id='run-c635cfc4-2631-4ba8-ab9b-1e94e613af63-0')]
The tool result is: fifty-five


In [45]:
output

{'input': 'give me a random number and them write in words and make it lower case',
 'chat_history': [],
 'agent_outcome': AgentFinish(return_values={'output': 'The random number is 55, and in words, it is "fifty-five".'}, log='The random number is 55, and in words, it is "fifty-five".'),
 'intermediate_steps': [(AgentActionMessageLog(tool='random_number', tool_input={'input': 'random number'}, log="\nInvoking: `random_number` with `{'input': 'random number'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"input":"random number"}', 'name': 'random_number'}}, response_metadata={'finish_reason': 'function_call'}, id='run-e1ece6be-900d-4eef-bff7-7063e0a0e947-0')]),
   '55'),
  (AgentActionMessageLog(tool='lower_case', tool_input={'input': 'FIFTY-FIVE'}, log="\nInvoking: `lower_case` with `{'input': 'FIFTY-FIVE'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"input":"FIFTY-FIVE"}', 'name': 'l